#### The EfficentNet model

Using effent because it should later run on edge AI device.

In [ ]:
########### Basic Parameters for Running: ################################
    
Training_Percentage = 0.2           
Epoch_Anz = 50
nb_classes = 100                     # move to 1. step
#input_shape = (32, 20,1)            # will be calculated
Batch_Size = 32
use_grayscale = True                # grayscale or rgb
model_filename = "eff100-gray.h5"

##########################################################################


In [ ]:
from keras.models import Model
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Conv2D, MaxPool2D, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, Rescaling, LayerNormalization

def get_post(x_in):
    x = LeakyReLU()(x_in)
    x = BatchNormalization()(x)
    return x

def get_block(x_in, ch_in, ch_out):
    x = Conv2D(ch_in,
               kernel_size=(1, 1),
               padding='same',
               use_bias=False)(x_in)
    x = get_post(x)

    x = DepthwiseConv2D(kernel_size=(1, 3), padding='same', use_bias=False)(x)
    x = get_post(x)
    x = MaxPool2D(pool_size=(2, 1),
                  strides=(2, 1))(x) # Separable pooling

    x = DepthwiseConv2D(kernel_size=(3, 1),
                        padding='same',
                        use_bias=False)(x)
    x = get_post(x)

    x = Conv2D(ch_out,
               kernel_size=(2, 1),
               strides=(1, 2),
               padding='same',
               use_bias=False)(x)
    x = get_post(x)

    return x


def Effnet(input_shape, nb_classes, include_top=True, weights=None, activation_top=None):
    x_in = Input(shape=input_shape)

    x = get_block(x_in, 32, 64)
    x = get_block(x, 64, 128)
    x = get_block(x, 128, 256)

    if include_top:
        x = Flatten()(x)
        #x = Dense(256, activation="relu")(x)
        #x = Dense(256, activation="relu")(x)
        x = Dense(nb_classes, activation=activation_top)(x)

    model = Model(inputs=x_in, outputs=x)

    if weights is not None:
        model.load_weights(weights, by_name=True)

    return model



### Utils Scripts

Various utility scripts now like label-encoding for sin/cos output.

In [ ]:
import tensorflow.keras as keras
import math
import numpy as np

def class_encoding(y_train, nb_classes):
    ret = np.zeros((len(y_train), nb_classes))
    for i, y in enumerate(y_train):
        #print(i, y, int((y*10)))
        ret[i, int((y*10))] = 1
    return ret

def class_decoding(y_train, nb_classes=100):
    ret = np.zeros((len(y_train), 1))
    for i, y in enumerate(y_train):
        ret[i] = (np.argmax(y))/10
    return ret

#### Augmentation functions

In [ ]:
import cv2
import numpy as np
import random
from scipy.stats import norm
import cv2
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator



def invert(imagem):
    if (random.getrandbits(1)):
        return (255)-imagem
    else:
        return imagem
        

Shift_Range = 0#2 # px
Brightness_Range = [0.4,0.9]
Rotation_Angle = 5
ZoomRange_Out = 0#0.2
ZoomRange_In = 0#0.1
ShearRange= 2
Channel_Shift=0#20
Blur = 5

def augmentation(x, y, Batch_Size = 32):
    datagen = ImageDataGenerator(width_shift_range=Shift_Range, 
                             height_shift_range=Shift_Range,
                             brightness_range=Brightness_Range,
                             zoom_range=[1-ZoomRange_In, 1+ZoomRange_Out],
                             rotation_range=Rotation_Angle,
                             #channel_shift_range=Channel_Shift,
                             fill_mode='nearest',
                             shear_range=ShearRange
                             ,preprocessing_function=invert
                             ,dtype=float
                             ,rescale=1./255)
    return datagen.flow(x, y, batch_size=Batch_Size)
    


#### Plot functions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_dataset(images, labels, columns=12, rows=5):

    fig = plt.figure(figsize=(18, 10))
    columns = 12
    rows = 5

    for i in range(1, columns*rows +1):
        if (i>len(labels)):
            break
        fig.add_subplot(rows, columns, i)
        plt.axis("off")
        plt.title(labels[i-1])  # set title
        if(images[i-1].shape[-1]==1):
            plt.imshow((images[i-1]), aspect='auto', cmap='gray')
        else:
            plt.imshow((images[i-1]), aspect='auto')
        plt.tight_layout()
    plt.show()

def plot_dataset_it(data_iter, columns=12, rows=5):

    fig = plt.figure(figsize=(18, 10))
    columns = 12
    rows = 5

    for i in range(1, columns*rows +1):
        img, label = data_iter.next()
        fig.add_subplot(rows, columns, i)
        plt.axis("off")
        plt.title(str(class_decoding(label[0].reshape(-1, nb_classes), nb_classes).reshape(-1)))  # set title
        if(img[0].shape[-1]==1):
            plt.imshow(img[0], aspect='auto', cmap='gray')
        else:
            plt.imshow(img[0], aspect='auto')
    plt.show()

def plot_acc_loss(history, modelname="modelname"):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(modelname)
    fig.set_figwidth(15)

    if "loss" in history.history:
        ax1.plot(history.history['loss'])
    if "accuracy" in history.history:
        ax2.plot(history.history['accuracy'])
    if "val_loss" in history.history:
        ax1.plot(history.history['val_loss'])
    if "val_accuracy" in history.history:
        ax2.plot(history.history['val_accuracy'])
    if "student_loss" in history.history:
        ax1.plot(history.history['student_loss'])
    if "sparse_categorical_accuracy" in history.history:
        ax2.plot(history.history['sparse_categorical_accuracy'])
    if "val_sparse_categorical_accuracy" in history.history:
        ax2.plot(history.history['val_sparse_categorical_accuracy'])
    if "student_accuracy" in history.history:
        ax2.plot(history.history['student_accuracy'])
    if "val_student_accuracy" in history.history:
        ax2.plot(history.history['val_student_accuracy'])
    if "distillation_loss" in history.history:
        ax1.plot(history.history['distillation_loss'])

    ax1.set_title('model loss')
    ax1.set_ylabel('loss')
    ax1.set_xlabel('epoch')
    ax2.set_ylabel('accuracy')
    ax2.set_xlabel('epoch')
    ax1.legend(['train','eval'], loc='upper left')
    axes = plt.gca()
    axes.set_ylim([0.92,1])
    plt.show()


def plot_divergence(divergationset, title1, nb_classes):
    fig = plt.figure(figsize=(40, 10))
    fig.suptitle(title1)
    plt.bar(np.arange (0, nb_classes/10, 0.1), divergationset, width=0.09, align='center')
    plt.ylabel('count')
    plt.xlabel('digit class')
    plt.xticks(np.arange(0, nb_classes/10, 0.1))
    return fig


def confusion_matrix(predicted, y_test, nb_classes):
    ytrue = pd.Series(np.argmax(class_encoding(y_test, nb_classes), axis = 1), name = 'actual')
    ypred = pd.Series(np.argmax(class_encoding(predicted, nb_classes), axis = 1), name = 'pred')
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    return pd.crosstab(ytrue, ypred)

#### Data loading function

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

def tmnist_percentals(use_grayscale=True):
    y_train = np.empty((0,1))
    if (use_grayscale):
        x_train = np.empty((0, 32, 20,1))
    else:
        x_train = np.empty((0, 32, 20,3))
    
    blacklist_url = "datasets/font-blacklist.txt"
    blacklist_data = pd.read_csv(blacklist_url, index_col=False)
    #print(blacklist_data.values.reshape(-1))
    
    for i in range(28):

        dataset_url = "datasets/TMNIST_PERCENTAL_"+str((i+1)*1000)+"_Data.csv"
        data = pd.read_csv(dataset_url, index_col=False)
        data = data.drop(data[data['names'].isin(blacklist_data.values.reshape(-1))].index)
        #print(data.head())
        y_tmnist = data[['labels']]
        #print(y_tmnist[y_tmnist.labels>10])
        X = data.drop({'labels','names'},axis=1)

        #print(data)
        X_images = (255-(X.values.reshape(-1,28,28,1)))
        X_images = tf.image.resize(X_images, (32,20))
        X_images = np.array(X_images).reshape(-1,32,20,1)
        X_images = X_images/255.
            
        if (use_grayscale!=True):
            X_images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(X_images))
            

        x_train = np.concatenate((X_images, x_train))
        y_train = np.concatenate((y_tmnist.values, y_train))
    #print(">10", y_train[y_train>10])
    return shuffle(x_train,  y_train, n_samples=len(y_train))



In [ ]:
import os
from PIL import Image 

def ziffer_data_files(input_dir='images'):
    imgfiles = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if (file.endswith(".jpg") and not file.startswith("10_")):
                #print(root + "/" + file)
                imgfiles.append(root + "/" + file)
    return  imgfiles

def ziffer_data(x_data, y_data, nb_classes, use_grayscale=True):

    files = ziffer_data_files("images")
    files = files + ziffer_data_files("ziffer_raw")
    #print("Ziffer Imagecount: ", len(files))
    y_data = np.array(y_data).reshape(-1,1)
    if (use_grayscale):
        x_data = np.array(x_data).reshape(-1,32,20,1)
    else: 
        x_data = np.array(x_data).reshape(-1,32,20,3)

    for aktfile in files:
        base = os.path.basename(aktfile)
        target = base[0:1]
        if target == "N":
            category = 10                # NaN does not work --> convert to 10

        else:
            category = int(target)
        test_image = Image.open(aktfile).resize((20, 32))
        if (use_grayscale):
            test_image = test_image.convert('L')
        test_image = np.array(test_image, dtype="float32")
        test_image = test_image/255.
            
        #print(test_image.shape)
        if (use_grayscale):
            test_image = test_image.reshape(1,32,20,1)
        else:
            test_image = test_image.reshape(1,32,20,3)

        # ignore the category 10
        if ( category<10):
            x_data = np.vstack((x_data, test_image))
            y_data = np.vstack((y_data, [category]))
    print("Ziffer data count: ", len(y_data))   
    return shuffle(x_data, y_data)




#### 

### Load data

In [ ]:
xz_data, yz_data = ziffer_data([],[], 10, use_grayscale=use_grayscale)

x_data, y_data = tmnist_percentals(use_grayscale=use_grayscale)
x_data = np.concatenate((x_data, xz_data))
y_data = np.concatenate((y_data, yz_data))
x_data, y_data = shuffle(x_data, y_data)

input_shape=x_data[0].shape
print(f"dataset x_data-size={len(x_data)}")
plot_dataset(x_data, y_data)

y_data = class_encoding(y_data, nb_classes)

### Augmentation

So it looks more like real meter digits

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Split train and validation data 
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=Training_Percentage)

train_iterator = augmentation(x_train, y_train)
validation_iterator = augmentation(x_test, y_test)

plot_dataset_it(train_iterator)       


### The Model

Is an small EfficientNet like model with 3 Dense layer at top. The last layer uses **tanh** as activation.

In [ ]:
import tensorflow as tf

model = Effnet(input_shape, nb_classes, activation_top=None)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              optimizer="adam", metrics = ["accuracy"])

In [ ]:
history = model.fit(train_iterator, 
                validation_data=validation_iterator, 
                batch_size=Batch_Size, 
                epochs = 50,
                verbose=1)

plot_acc_loss(history, "Trainingsresults")

model.save(model_filename)

### Verfify the training

Now a few (5000) images from validation set will be used to evaluate the model.

All the first (max 30) false predicted images will be shown.

If all fine the predictions will be shown in a diagram and it should be a sinus and cosinus wave. The glitches are the false predictions.

In [ ]:
import numpy as np
from operator import itemgetter

res = []
stat_Anz = []
stat_Abweichung = []
false_img = []
false_label = []
test_count = 5000

for i in range(100):
    stat_Anz.append(0)
    stat_Abweichung.append(0)

for x_test, y_test in validation_iterator:
    if (test_count<0):
        break
    test_count = test_count - Batch_Size
           
    for (x, y) in zip(x_test, y_test):
            
        target = class_decoding([y])
        classes = model.predict(np.expand_dims(x.reshape(input_shape), axis=0))
        
        
        out_target = class_decoding(classes)[0]

                       
        if target != out_target:
            lbl_text = "Expected: " + str(target) + "\n Predicted: " + str(out_target)
            print("Exp: " + str(target) + " Pred: " + str(out_target))
            false_img.append(x)
            false_label.append(lbl_text)
    
for i in range(100):
    if (stat_Anz[i] != 0):
        stat_Abweichung[i] = stat_Abweichung[i] / stat_Anz[i]

res = np.asarray(res)
res_step_1 = res
print(f"{len(false_label)} false predicted")
plot_dataset(np.array(false_img), false_label, columns=11, rows=3)



In [ ]:
import numpy as np
import math
from operator import itemgetter

res = []
stat_Abweichung = np.zeros(nb_classes)
false_img = []
false_label = []
max_delta = 0.11

def isclose(a, b, delta):
    return min(abs(a-b),abs(a-(10-b)))<=delta


for x, y in zip(xz_data, yz_data):

    target = y
    classes = model.predict(np.expand_dims(x.reshape(input_shape), axis=0))
    out_target = class_decoding(classes)[0]
    res.append(out_target)
                   
    if not isclose(target, out_target, max_delta) :
        lbl_text = "Expected: " + str(target) + "\n Predicted: " + str(out_target)
        #print("Exp: " + str(target) + " Pred: " + str(out_target))
        false_img.append(x)
        false_label.append(lbl_text)
        stat_Abweichung[int(abs(target-out_target)*10)] = stat_Abweichung[int(abs(target-out_target)*10)]+1

res = np.asarray(res)
res_step_1 = res
print(f"Tested images: {len(yz_data)}. {len(false_label)} false predicted. Accuracy is: {1-len(false_label)/len(yz_data)}")
plot_divergence(stat_Abweichung, "Divergation of false predicted", nb_classes)
print(confusion_matrix(res, yz_data, nb_classes))
plot_dataset(np.array(false_img), false_label, columns=11, rows=3)